# Lung and Colon Cancer Detection
This notebook demonstrates the process of building a machine learning model to detect lung and colon cancer using histopathological images. The dataset contains labeled images of cancerous and non-cancerous (<em>healthy</em>) tissues. 

There are five classes in this dataset: 
- Lung benign tissue (<em>healthy</em>)
- Lung adenocarcinoma
- Lung squamos cell carcinoma
- Colon adenocarcinoma
- Colong benign tissue (<em>healthy</em>)

The goal is to compare different Convolutional Neural Networks (CNNs) to explore the strengths and weaknesses of various architectures and understand which ones perform best for the chosen dataset. Ultimately, the most robust classifier (CNN) will be identified and can accurately identify cancerous lung or colon tissues from the given sample images. 

For a fair comparison of the different CNNs, it is necessary to set some guidelines / rules:
- The dataset has to be properly preprocessed.
- The same training parameters are used 
  - Learning rate
  - Batch size
  - Number of epochs
- The same optimizer is used
  - Isolates the effect if the CNN architecture on performance

Those rules will ensure that the comparison is consistent, controlled and fair.

The dataset used in this notebook is sourced from Kaggle: https://www.kaggle.com/datasets/andrewmvd/lung-and-colon-cancer-histopathological-images/data

## Required Packages

In [2]:
import os
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

## Dataset Handling
### Import Dataset
The dataset containes 25.000 images.

In [ ]:
lung_dataset = '../lung_colon_image_set/lung_image_sets'
colon_dataset = '../lung_colon_image_set/colon_image_sets'

if not os.path.exists(lung_dataset):
    raise FileNotFoundError(f"Dataset path '{lung_dataset}' does not exist!")
if not os.path.exists(colon_dataset):
    raise FileNotFoundError(f"Dataset path '{colon_dataset}' does not exist!")

lung_classes = sorted(os.listdir(lung_dataset))
colon_classes = sorted(os.listdir(colon_dataset))

print(f"Classes found in lung dataset: {lung_classes}")
print(f"Classes found in colon dataset: {colon_classes}")

lc_classes = sorted(set(lung_classes + colon_classes))
print(f"Lung and Colon classes combined: {lc_classes}")

### Split Dataset
The dataset is split into the following three categories with pre defined percentages:
- Training data (<em>80 %</em>)
- Validation data (<em>10 %</em>)
- Testing data (<em>10 %</em>)

In [ ]:
def prepare_splits(data_directories, split_ratios=(0.8, 0.1, 0.1)):
    all_data = {}
    for data_directory in data_directories:
        for class_name in sorted(os.listdir(data_directory)):
            class_directory = os.path.join(data_directory, class_name)
            if os.path.isdir(class_directory):
                all_data.setdefault(class_name, []).extend(os.path.join(class_directory, file_name) for file_name in os.listdir(class_directory))

    for class_name, files in all_data.items():
        print(f"Class '{class_name}' has {len(files)} images")

    dataset_splits = {'training': [], 'validation': [], 'testing': []}

    for class_name, files in all_data.items():
        training_dataset, temporary_dataset = train_test_split(files, test_size=(1 - split_ratios[0]), random_state=42)

        validation_dataset, testing_dataset = train_test_split(temporary_dataset, test_size=split_ratios[2]/(split_ratios[1] + split_ratios[2]), random_state=42)

        dataset_splits['training'].extend(training_dataset)
        dataset_splits['validation'].extend(validation_dataset)
        dataset_splits['testing'].extend(testing_dataset)

    return dataset_splits

dataset_directories = [lung_dataset, colon_dataset]
dataset_splits = prepare_splits(dataset_directories)

print(f"Training dataset has {len(dataset_splits['training'])} images")
print(f"Validation dataset has {len(dataset_splits['validation'])} images")
print(f"Testing dataset has {len(dataset_splits['testing'])} images")

### Dataset Class
Initializes the dataset class.

In [4]:
class CancerDetectionDataset(Dataset):
    def __init__(self, file_paths, all_classes, transform=None):
        self.file_paths = file_paths
        self.all_classes = all_classes
        self.labels = [all_classes.index(os.path.basename(os.path.dirname(file_path))) for file_path in file_paths]
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        image_path = self.file_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        return image, label

## Mean and Standard Deviation
Overall, normalization and standardization helps in stabilizing and speeding up the training process of machine learning models.

These are the main reasons:
- Subtracting the mean from each image centers the data around zero.
- Dividing by the standard deviation scales the data to have unit variance.
- Normalized inputs can lead to faster and improved convergance during training because the gradients are more stable and the optimization is more efficient.
- Normalization ensures that all input features have are within the same, consistent range.

Calculating the mean and standard deviation of the specific dataset, rather than using standard values, is quite helpful:
- Dataset specificity,
- Improved model performance,
- Avoiding bias and
- Consistency

By calculating the mean and standard deviation specific to the dataset, it is ensured that the normalization is optimal for the data, leading to better model performance and more reliable results.

This step will usually be performed once to determine the mean and standard deviation.

In [ ]:
def get_image_paths(dataset_splits):
    keys = ['training', 'validation', 'testing']
    image_paths = []

    for key in keys:
        image_paths.extend(dataset_splits[key])
    
    return image_paths

transformms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=(-45, 45)),
    transforms.ToTensor()
])

def calculate_mean_std(loader):
    mean = torch.zeros(3)
    std = torch.zeros(3)
    total_images_count = 0
    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += batch_samples

    mean /= total_images_count
    std /= total_images_count
    
    return mean, std

if __name__ == '__main__':
    dataset = CancerDetectionDataset(get_image_paths(dataset_splits), lc_classes, transform=transformms)
    loader = DataLoader(dataset, batch_size=64, shuffle=False)

    mean, std = calculate_mean_std(loader)
    print(f'Mean: {torch.round(mean, decimals=3)}')
    print(f'Std: {torch.round(std, decimals=3)}')

## Transformer
Transforms the images.

In [70]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=(-45, 45)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.644, 0.529, 0.774], std=[0.262, 0.252, 0.280])
])

## Dataset Classes und Data Loaders
Builds the dataset classes and data loadersfor the training, validation and test data.

In [ ]:
training_dataset = CancerDetectionDataset(dataset_splits['training'], lc_classes, transform=transform)
validation_dataset = CancerDetectionDataset(dataset_splits['validation'], lc_classes, transform=transform)
testing_dataset = CancerDetectionDataset(dataset_splits['testing'], lc_classes, transform=transform)

training_loader = DataLoader(training_dataset, batch_size=16, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=16, shuffle=False)
testing_loader = DataLoader(testing_dataset, batch_size=16, shuffle=False)

## GPU Initialization


In [ ]:
device = torch.device('mps' if torch.mps.is_available() else 'cpu')
print(f"MPS available: {torch.backends.mps.is_available()}")
print(f"Using device: {device}")

## CNN-Models
The starting point for this experiment is the deep convolutional neural network model 'VGG-16'. This model is composed of 16 layers (13 convolutional layers & 3 fully connected layers). VVG16 is known for it's simplicity and depth. Furthermore, the model is often used as a benchmark.

The '__init__' constructur initializes the layers of the VGG-16 model. Convolutional layers are with batch normalization and ReLU activation functions. Max pooling is applied after layers 2, 4, 7 and 10 to reduce spatial dimensions. The fully connected layers use dropout to prevent overfitting and ReLU activation functions.

The 'forward' method defines the forward pass of the model. <br>
out = self.layer1(x) <br> 
-> passes the input x through the first convolutional layer (layer1).out = self.layer2out) <br> 
-> passes the output of the first layer trough the second convolutional layer (layer2) ... <br>
out = out.reshape(out.size(0), -1) -> flattens the output from convolutional layers to a 1D tensor and passes the flattened output through the fully connected layers.

In [ ]:
class VGG16(nn.Module):
    def __init__(self, num_classes=5):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2 = nn.Sequential(
            nn.Linear(4096, num_classes))
        

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

## Model Training

In [ ]:
def train_model(model, training_loader, validation_loader, number_epochs = 5, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    model.to(device)

    training_losses = []
    validation_losses = []
    validation_accuracies = []

    for epoch in range(number_epochs):
        model.train()
        running_loss = 0.0

        for images, labels in training_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        epoch_training_loss = running_loss / len(training_loader)
        training_losses.append(epoch_training_loss)

        print(f"Epoch {epoch + 1}/{number_epochs}, Training Loss: {running_loss/len(training_loader)}")

        
        model.eval()
        validation_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        with torch.no_grad():
            for images, labels in validation_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                validation_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_predictions += labels.size(0)
                correct_predictions += (predicted == labels).sum().item()
        
        epoch_validation_loss = validation_loss / len(validation_loader)
        validation_losses.append(epoch_validation_loss)
        validation_accuracies.append(validation_accuracy)

        validation_accuracy = 100 * correct_predictions / total_predictions

        print(f"Validation Loss: {validation_loss/len(validation_loader)}, Validation Accuracy: {validation_accuracy}")

    return training_losses, validation_losses, validation_accuracies


In [ ]:
if __name__ == "__main__":
    model = VGG16(num_classes=len(lc_classes)).to(device)
    training_losses, validation_losses, validation_accuracies = train_model(model, training_loader, validation_loader)

    print("Training completed!")